<a href="https://colab.research.google.com/github/ARD7989/AN-INTEGRATED-APPROACH-FOR-THE-DETECTION-OF-RETINAL-DISEASES-USING-CNN/blob/main/MAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy import signal
import cv2
import sys
import pandas as pd
import os
import math
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
from tensorflow.keras.models import Model, load_model


def segment(image):
    image = image[400:1400]


    B,G,R = cv2.split(image)
    #print("\n \n Green Channel")
    #cv2_imshow(G)
    #print("\n \n RED Channel")
    #cv2_imshow(R)

    G = clahe.apply(G)
    R = clahe.apply(R)
    M = 100
    filter = signal.gaussian(M, std=6)
    STDf = filter.std()

    Ar = R - R.mean() - R.std()
    Mr = Ar.mean()
    SDr = Ar.std()
    Thr = 0.5*M - STDf - Ar.std()
    #print('\n \n Threshold Value of disc :',Thr)

    M = 33
    filter_cup = signal.gaussian(M,std=6)
    STDg = filter_cup.std()

    Ag = G - G.mean() - G.std()
    Mg = Ag.mean()
    SDg = Ag.std()
    Thg = 0.5*M +2*STDg + 2*SDg + Mg
    #print('\n \n Threshold Value of cup :',Thg)

    hist,bins = np.histogram(Ag.ravel(),256,[0,256])
    histr,binsr = np.histogram(Ar.ravel(),256,[0,256])

    smooth_hist_g=np.convolve(filter_cup,hist)
    smooth_hist_r=np.convolve(filter,histr)

    r,c = Ag.shape
    Od = np.zeros(shape=(r,c))
    Oc = np.zeros(shape=(r,c))

    for i in range(1,r):
        for j in range(1,c):
            if Ar[i,j]>Thr:
                Od[i,j]=255
            else:
                Od[i,j]=0
    for i in range(1,r):
        for j in range(1,c):

            if Ag[i,j]>Thg:
                Oc[i,j]=255
            else:
                Oc[i,j]=0

    plt.imsave('disc.png',Od)
    plt.imsave('cup.png',Oc)

    disc = cv2.imread('/content/disc.png')
    #print("\n \n Binary Image disc")
    #cv2_imshow(disc)

    cup = cv2.imread('/content/cup.png')
    #print("\n \n Binary Image cup")
    #cv2_imshow(cup)

# FUNCTION TO CALCULATE CDR
def cdr(cup,disc):

    R1 = cv2.morphologyEx(cup, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(19,19)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(29,29)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(37,37)), iterations = 1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(47,47)), iterations = 1)
    img = r3
    ret,thresh = cv2.threshold(img,127,255,0)

    #print("\n \n CUP Morphology")
    #cv2_imshow(img)

    im_bw = cv2.cvtColor(thresh, cv2.COLOR_RGB2GRAY)
    contours,hierarchy = cv2.findContours(im_bw, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cup_diameter = 0
    largest_area = 0
    ellipse_cup = contours[0]
    if len(contours) != 0:
        for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv2.contourArea(contours[i])
                if (area>largest_area):
                    largest_area=area
                    index = i
                    ellipse_cup = cv2.fitEllipse(contours[i])

    cv2.ellipse(img,ellipse_cup,(0,0,255),3)
    x,y,w,h = cv2.boundingRect(contours[index])
    cup_diameter = max(w,h)

    #print("\n \n Recognizing CUP")
    #cv2_imshow(img)

    R1 = cv2.morphologyEx(disc, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(19,19)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(29,29)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(37,37)), iterations = 1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(47,47)), iterations = 1)
    img2 = r3
    #print("\n \n disc Morphology")
    #cv2_imshow(img2)

    ret,thresh = cv2.threshold(img2,127,255,0)
    im_bw = cv2.cvtColor(thresh, cv2.COLOR_RGB2GRAY)
    contours,hierarchy = cv2.findContours(im_bw, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    disc_diameter = 0
    largest_area = 0
    ellipse_disc = ellipse_cup
    if len(contours) != 0:
          for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv2.contourArea(contours[i])
                if (area>largest_area):
                    largest_area=area
                    index = i
                    ellipse_disc = cv2.fitEllipse(contours[i])

    cv2.ellipse(img2,ellipse_disc,(0,0,255),3)
    x,y,w,h = cv2.boundingRect(contours[index])
    disc_diameter = max(w,h)

    #print("\n \n Recognizing disc")
    #cv2_imshow(img2)


    cdr = cup_diameter/disc_diameter
    return cdr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
image = files.upload()

Saving drishtiGS_059.png to drishtiGS_059.png


In [ ]:

set_path = os.path.join('drishtiGS_059.png')
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(10,10))
image = cv2.imread(set_path,1)
segment(image)
cup = cv2.imread('/content/cup.png')
disc = cv2.imread('/content/disc.png')
cdr_cal = cdr(cup,disc)
inception_resnet_v2_glaucoma = load_model('/content/drive/MyDrive/inception_resnet_v2_glaucoma.h5')

img = cv2.resize(image, (224, 224))

transformed_glaucoma = cv2.applyColorMap(img, cv2.COLORMAP_HSV)
transformed_glaucoma = cv2.resize(transformed_glaucoma, (224, 224))

image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))
image = np.array(image) / 255
image = np.expand_dims(image, axis=0)

gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(gray_image,100,300,0)
contours,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contoured_glaucoma = cv2.drawContours(img,contours,-1,(0,300,0),1)
contoured_glaucoma = cv2.resize(contoured_glaucoma, (224, 224))

inception_resnet_v2_glaucoma_pred = inception_resnet_v2_glaucoma.predict(image)
probability = inception_resnet_v2_glaucoma_pred[0]
print("inception_resnet_v2_glaucoma Predictions:")
if probability[0] > 0.5:
    inception_resnet_v2_glaucoma_glaucoma_pred = str(' Glaucoma POSITIVE')
else:
    inception_resnet_v2_glaucoma_glaucoma_pred = str( ' Glaucoma NEGATIVE')
print(inception_resnet_v2_glaucoma_glaucoma_pred)
print('\n \n CUP TO DISC RATIO : ',cdr_cal)

1/1 [==============================] - 3s 3s/step
inception_resnet_v2_glaucoma Predictions:
 Glaucoma POSITIVE

 
 CUP TO DISC RATIO :  0.7628571428571429
